In [76]:
import folium
import pandas as pd
import numpy as np
from collections import namedtuple

# define the world map
world_map = folium.Map()

# display world map
world_map

latitude = 12.6985352
longitude = 99.7649053

# Create map and display it
san_map = folium.Map(location=[latitude, longitude], zoom_start=6)

data2=pd.read_csv(r"D:\Onedrive\68，地理信息\15hub.csv")
# add pop-up text to each marker on the map
latitudes = list(data2.lat)
longitudes = list(data2.lng)
labels = list(data2.name)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(san_map) 

In [77]:
data3=pd.read_excel(r"D:\Onedrive\68，地理信息\HUB间装载率地图\HUB间装载率信息.xlsx")
weizhi=[]
for x in data3.index:
    a=[[data3.lat1.loc[x],data3.lng1.loc[x]],[data3.lat2.loc[x],data3.lng2.loc[x]]]
    weizhi.append(a)
colorr=[]
for y in data3.index:
    b=[data3.color.loc[y]]
    colorr.append(b)
weightt=[]
for z in data3.index:
    c=[data3.weight.loc[z]]
    weightt.append(c)

In [78]:
weizhi

[[[14.930258, 102.132909], [13.651285, 100.648434]],
 [[9.118142, 99.292152], [13.651285, 100.648434]],
 [[14.930258, 102.132909], [9.118142, 99.292152]],
 [[15.632067, 100.127893], [13.651285, 100.648434]],
 [[14.930258, 102.132909], [15.632067, 100.127893]],
 [[9.118142, 99.292152], [15.632067, 100.127893]],
 [[14.930258, 102.132909], [13.650154, 100.347259]],
 [[9.118142, 99.292152], [13.650154, 100.347259]],
 [[15.632067, 100.127893], [13.650154, 100.347259]],
 [[15.632067, 100.127893], [18.179995, 99.410626]],
 [[13.650154, 100.347259], [18.179995, 99.410626]],
 [[18.742318, 99.007807], [13.651285, 100.648434]],
 [[15.632067, 100.127893], [18.742318, 99.007807]],
 [[13.650154, 100.347259], [18.742318, 99.007807]],
 [[15.632067, 100.127893], [19.866592, 99.844869]],
 [[13.650154, 100.347259], [19.866592, 99.844869]],
 [[18.179995, 99.410626], [19.866592, 99.844869]],
 [[14.930258, 102.132909], [16.400216, 102.871005]],
 [[13.650154, 100.347259], [16.400216, 102.871005]],
 [[17.1248

In [79]:
def get_arrows(locations, color='blue', size=6, n_arrows=3):
    
    '''
    Get a list of correctly placed and rotated 
    arrows/markers to be plotted
    
    Parameters
    locations : list of lists of lat lons that represent the 
                start and end of the line. 
                eg [[41.1132, -96.1993],[41.3810, -95.8021]]
    arrow_color : default is 'blue'
    size : default is 6
    n_arrows : number of arrows to create.  default is 3
    Return
    list of arrows/markers
    '''
    
    Point = namedtuple('Point', field_names=['lat', 'lon'])
    
    # creating point from our Point named tuple
    p1 = Point(locations[0][0], locations[0][1])
    p2 = Point(locations[1][0], locations[1][1])
    
    # getting the rotation needed for our marker.  
    # Subtracting 90 to account for the marker's orientation
    # of due East(get_bearing returns North)
    rotation = get_bearing(p1, p2) - 90
    
    # get an evenly space list of lats and lons for our arrows
    # note that I'm discarding the first and last for aesthetics
    # as I'm using markers to denote the start and end
    arrow_lats = np.linspace(p1.lat, p2.lat, n_arrows + 2)[1:n_arrows+1]
    arrow_lons = np.linspace(p1.lon, p2.lon, n_arrows + 2)[1:n_arrows+1]
    
    arrows = []
    
    #creating each "arrow" and appending them to our arrows list
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(folium.RegularPolygonMarker(location=points, 
                      fill_color=color, number_of_sides=3, 
                      radius=size, rotation=rotation))
    return arrows

In [80]:
def get_bearing(p1, p2):
    
    '''
    Returns compass bearing from p1 to p2
    
    Parameters
    p1 : namedtuple with lat lon
    p2 : namedtuple with lat lon
    
    Return
    compass bearing of type float
    
    Notes
    Based on https://gist.github.com/jeromer/2005586
    '''
    
    long_diff = np.radians(p2.lon - p1.lon)
    
    lat1 = np.radians(p1.lat)
    lat2 = np.radians(p2.lat)
    
    x = np.sin(long_diff) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2) 
        - (np.sin(lat1) * np.cos(lat2) 
        * np.cos(long_diff)))
    bearing = np.degrees(np.arctan2(x, y))
    
    # adjusting for compass bearing
    if bearing < 0:
        return bearing + 360
    return bearing

In [81]:
len(weizhi)

55

In [82]:
aaa=list(range(0,55,1))

In [83]:
for x in aaa:
    a=get_arrows(locations=weizhi[x], n_arrows=3)
    for y in [0,1,2]:
        a[y].add_to(san_map)

san_map

In [85]:
for weizhi,colorr,weightt in zip(weizhi,colorr,weightt):
    folium.PolyLine(weizhi,color=colorr, weight=weightt, opacity=0.5).add_to(san_map)
    
san_map

In [86]:
san_map.save("./lines_location_change.html")